In [2]:
import numpy as np
import pandas as pd

# Calculate Low-Income Thresholds

In [2]:
# puma codes for alameda, contra costa, and san francisco counties
alameda_puma = np.arange(101, 111)
cc_puma = np.arange(1301, 1310)
sf_puma = np.arange(7501, 7508)

In [3]:
def extend_low_income(thres, upper_N=16):
    extras = (np.arange(9, upper_N) - 8) * thres[3] * 0.08 + thres[-1]
    thres = np.append(thres, extras)
    return thres

In [4]:
# alameda, contra costa, and sf low income thresholds per number of persons in household 
alameda_low_income = np.array([62750, 71700, 80650, 89600, 96800, 103950, 111150, 118300])
cc_low_income = np.array([62750, 71700, 80650, 89600, 96800, 103950, 111150, 118300])
sf_low_income = np.array([82200, 93950, 105700, 117400, 126800, 136200, 145600, 155000])
# extend the low income thresholds for households of size >8
alameda_low_income = extend_low_income(alameda_low_income)
cc_low_income = extend_low_income(cc_low_income)
sf_low_income = extend_low_income(sf_low_income)
# store in list
low_income_thresholds = [alameda_low_income, cc_low_income, sf_low_income]

# Consolidate Low-Income Tables

In [3]:
# read in household PUMS data
hca = pd.read_csv('ss16hca.csv')

In [10]:
# extract samples corresponding to each county
alameda = hca.loc[hca.PUMA.isin(alameda_puma)]
cc = hca.loc[hca.PUMA.isin(cc_puma)]
sf = hca.loc[hca.PUMA.isin(sf_puma)]
# store tables in list
tables = [alameda, cc, sf]

In [11]:
# extract low income portion of the table
low_income_tables = []
# use FINCP or HINCP?
for idx, table in enumerate(tables):
    # remove NaN household incomes
    table = table.loc[table.FINCP.notna()]
    low_income_threshold = low_income_thresholds[idx]
    low_income_table = table.loc[table.FINCP < low_income_threshold[table.NP - 1]]
    low_income_tables.append(low_income_table)

# Internet

In [37]:
def calculate_percent_internet(table, weighted=True):
    if weighted:
        weighted_sum = table.WGTP.sum()
        no_internet = table.loc[table.ACCESS == 3].WGTP.sum()
        percent_no_internet = 100 * no_internet/weighted_sum
    else:
        total = table.shape[0]
        no_internet = table.loc[table.ACCESS==3].shape[0]
        percent_no_internet = 100 * no_internet/total
    return percent_no_internet

In [38]:
alameda_internet = calculate_percent_internet(low_income_tables[0])
print('In Alameda County, %0.2f%% of low income households lack access to the Internet at home.' %alameda_internet)
sf_internet = calculate_percent_internet(low_income_tables[2])
print('In San Francisco County, %0.2f%% of low income households lack access to the Internet at home.' %sf_internet)
cc_internet = calculate_percent_internet(low_income_tables[1])
print('In Contra Costa County, %0.2f%% of low income households lack access to the Internet at home.' %cc_internet)

In Alameda County, 13.44% of low income households lack access to the Internet at home.
In San Francisco County, 10.73% of low income households lack access to the Internet at home.
In Contra Costa County, 8.65% of low income households lack access to the Internet at home.


# Computer

In [39]:
def calculate_percent_computer(table, weighted=True):
    if weighted:
        weighted_sum = table.WGTP.sum()
        no_computer = table.loc[table.LAPTOP == 2].WGTP.sum()
        percent_no_computer = 100 * no_computer/weighted_sum
    else:
        total = table.shape[0]
        no_computer = table.loc[table.LAPTOP == 2].shape[0]
        percent_no_computer = 100 * no_computer/total
    return percent_no_computer

In [40]:
alameda_computer = calculate_percent_computer(low_income_tables[0])
print('In Alameda County, %0.2f%% of low income households lack access to a computer at home.' %alameda_computer)
sf_computer = calculate_percent_computer(low_income_tables[2])
print('In San Francisco County, %0.2f%% of low income households lack access to a computer at home.' %sf_computer)
cc_computer = calculate_percent_computer(low_income_tables[1])
print('In Contra Costa County, %0.2f%% of low income households lack access to a computer at home.' %cc_computer)

In Alameda County, 23.21% of low income households lack access to a computer at home.
In San Francisco County, 18.10% of low income households lack access to a computer at home.
In Contra Costa County, 22.66% of low income households lack access to a computer at home.


In [29]:
low_income_tables[1].WGTP.sum()/cc.WGTP.sum()

0.2699299439649087

In [32]:
print(low_income_tables[0].WGTP.sum()/alameda.WGTP.sum())
print(low_income_tables[1].WGTP.sum()/cc.WGTP.sum())
print(low_income_tables[2].WGTP.sum()/sf.WGTP.sum())

0.24833772383207176
0.2699299439649087
0.2016455248292488


|| Alameda | San Francisco | Contra Costa |
|---|---------|----|--------------|
|<b>Lacks Internet Access</b> | 11.60% | 10.34% | 8.13% | 
|<b>Lacks Internet Access (Weighted)</b> | 13.44% | 10.73% | 8.65% | 
|<b>Lacks Computer Access</b> | 19.44% | 16.11% | 17.94% | 
|<b>Lacks Computer Access (Weighted)</b> | 23.21% | 18.10% | 22.66% | 